In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from pickle import dump

In [10]:
pairs = pd.read_csv('../input/fourpoints-location-matching/train_pairs.csv')
#pairs = pairs.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (9,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
cols = ['latitude_1','longitude_1','country_1','latitude_2','longitude_2','country_2','name_1_fuzzy', 'address_1_fuzzy', 'city_1_fuzzy',
       'state_1_fuzzy', 'url_1_fuzzy', 'categories_1_fuzzy']
ids = ['id_1','id_2']

In [12]:
train_id = pairs[ids]
train_X = pairs[cols]
train_X['country_same'] = np.where(train_X['country_1'] == train_X['country_2'],1,0)
del train_X['country_1'] 
del train_X['country_2']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
col_scaler = ['latitude_1','longitude_1','latitude_2','longitude_2']
X_transform = train_X[col_scaler]
scaler = RobustScaler().fit(X_transform)
dump(scaler, open('scaler.pkl', 'wb')) # save the scaler
train_X[col_scaler] = scaler.transform(X_transform)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [14]:
train_y = pairs['match']
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
              importance_type='gain',
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=2, n_estimators=400, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.9, tree_method='exact', validate_parameters=1,
              verbosity=None)
xgb.fit(train_X, train_y)
xgb.save_model("model.json")